In [1]:
import pandas as pd
import numpy as np
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import pandas as pd
import statsmodels.api as sm
from IPython.display import Image
from sklearn import linear_model, datasets
from sklearn.metrics import mean_squared_error
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import folium
import pylab 
import scipy.stats as stats
from forex_python.converter import CurrencyRates
pd.set_option('display.max_columns', 40)

C:\Users\Tao\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
c = CurrencyRates()
currenciesDict = {
    'BWP': 9.64, 
    'BRL': c.convert('USD', 'BRL', 1),
    'USD': c.convert('USD', 'USD', 1),
    'AED': 3.67, 
    'INR': c.convert('USD', 'INR', 1), 
    'IDR': c.convert('USD', 'IDR', 1), 
    'GBP': c.convert('USD', 'GBP', 1), 
    'QAR': 3.64, 
    'ZAR': c.convert('USD', 'ZAR', 1),
    'LKR': 155.04,
    'TRY': c.convert('USD', 'TRY', 1)
}
print(currenciesDict)
def convertCurrency(x):
    return x['Average Cost for two']/currenciesDict[x['Currency in short']]

{'BWP': 9.64, 'BRL': 3.4082, 'USD': 1.0, 'AED': 3.67, 'INR': 65.207, 'IDR': 13753.0, 'GBP': 0.70147, 'QAR': 3.64, 'ZAR': 12.053, 'LKR': 155.04, 'TRY': 4.0928}


In [3]:
df=pd.read_csv("zomato.csv",encoding = "ISO-8859-1",low_memory=False)
df.insert(loc=12,column="Currency in USD",value=0)
df.insert(loc=12,column="Currency in short",value=df["Currency"])
df["Currency in short"].replace("Botswana Pula(P)","BWP",inplace=True)
df["Currency in short"].replace("Brazilian Real(R$)","BRL",inplace=True)
df["Currency in short"].replace("Dollar($)","USD",inplace=True)
df["Currency in short"].replace("Emirati Diram(AED)","AED",inplace=True)
df["Currency in short"].replace("Indian Rupees(Rs.)","INR",inplace=True)
df["Currency in short"].replace("Indonesian Rupiah(IDR)","IDR",inplace=True)
df["Currency in short"].replace("NewZealand($)","USD",inplace=True)
df["Currency in short"].replace("Pounds(£)","GBP",inplace=True)
df["Currency in short"].replace("Qatari Rial(QR)","QAR",inplace=True)
df["Currency in short"].replace("Rand(R)","ZAR",inplace=True)
df["Currency in short"].replace("Sri Lankan Rupee(LKR)","LKR",inplace=True)
df["Currency in short"].replace("Turkish Lira(TL)","TRY",inplace=True)
df["Currency in short"].unique()

array(['BWP', 'BRL', 'USD', 'AED', 'INR', 'IDR', 'GBP', 'QAR', 'ZAR',
       'LKR', 'TRY'], dtype=object)

In [6]:
df['Currency in USD'] = df.apply(convertCurrency, axis=1)
print(df.columns.values)

['Restaurant ID' 'Restaurant Name' 'Country Code' 'City' 'Address'
 'Locality' 'Locality Verbose' 'Longitude' 'Latitude' 'Cuisines'
 'Average Cost for two' 'Currency' 'Currency in short' 'Currency in USD'
 'Has Table booking' 'Has Online delivery' 'Is delivering now'
 'Switch to order menu' 'Price range' 'Aggregate rating' 'Rating color'
 'Rating text' 'Votes']


In [80]:
df2 = df.copy()
cuisines = df2.Cuisines.str.split(',', expand=True).stack().str.strip().reset_index(level=1, drop=True)
df_cuisines = pd.DataFrame(data=cuisines, columns=["Cuisines"])
df2 = df2.drop(['Cuisines'], axis=1).join(df_cuisines).reset_index(drop=True)
single_cusine_data=(df2.groupby(['Cuisines'], as_index=False)['Restaurant ID'].count())
single_cusine_data['Mean Rating']=(df1.groupby(['Cuisines'], as_index=False)['Aggregate rating'].mean().round())['Aggregate rating']
single_cusine_data['Votes']=(df1.groupby(['Cuisines'], as_index=False)['Votes'].mean().round())['Votes']
single_cusine_data['Price']=(df1.groupby(['Cuisines'], as_index=False)['Currency in USD'].mean().round())['Currency in USD']

Top6=single_cusine_data.sort_values('Restaurant ID',ascending=False).iloc[:6,]
Top6

,Cuisines,Restaurant ID,Mean Rating,Votes,Price
100,North Indian,3960,3.0,151.0,11.0
34,Chinese,2735,3.0,133.0,11.0
49,Fast Food,1986,3.0,93.0,7.0
95,Mughlai,995,3.0,153.0,11.0
70,Italian,764,4.0,431.0,21.0
13,Bakery,745,2.0,77.0,7.0


In [88]:
Not_top6=single_cusine_data.sort_values('Restaurant ID',ascending=False).iloc[6:,]
Not_top6.loc['All'] = pd.Series(Not_top6['Restaurant ID'].sum(), index = ['Restaurant ID'])
Not_top6.loc['All',"Mean Rating"] = Not_top6["Mean Rating"].mean(axis=0)
Not_top6.loc['All',"Votes"] = Not_top6["Votes"].mean(axis=0)   
Not_top6.loc['All',"Price"] = Not_top6["Price"].mean(axis=0)
Not_top6.loc['All',"Cuisines"] ="Other"
Other_all=Not_top6.tail(1)

In [87]:
top6_all=pd.concat([Top6, Other_all.round()], axis=0)
top6_all

,Cuisines,Restaurant ID,Mean Rating,Votes,Price
100,North Indian,3960.0,3.0,151.0,11.0
34,Chinese,2735.0,3.0,133.0,11.0
49,Fast Food,1986.0,3.0,93.0,7.0
95,Mughlai,995.0,3.0,153.0,11.0
70,Italian,764.0,4.0,431.0,21.0
13,Bakery,745.0,2.0,77.0,7.0
All,Other,8525.0,4.0,357.0,30.0
